### The idea is to compare current spot price with min and max of the next LOOKAHEAD num of prices

In [1]:
import pandas as pd
import numpy as np
import math
import time

from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [2]:
vic_data = pd.read_csv('../../../data/victoria.csv')

In [3]:
# Constant Variables 
TIME = 'Time (UTC+10)'
PRICE = 'Regions VIC Trading Price ($/MWh)'
GENERATION = 'Regions VIC Trading Total Intermittent Generation (MW)'
DEMAND = 'Regions VIC Operational Demand (MW)'

POWER = 300
CAPACITY = 580
EFFICIENCY = 0.9
CHARGE_EFF = 90
DISCHARGE_EFF = 90
MLF = 0.991
FIXED_OP = 8.1
VAR_OP = 0

CHARGE = -1
DISCHARGE = 1

In [4]:
def create_df(ori_df):
    """ Returns a proper dataframe with columns needed """

    df = ori_df[[TIME, PRICE]]
    df['raw_power'] = 0
    df['dispatch'] = 0
    df['revenue'] = 0
    df['opening'] = 0
    df['closing'] = 0
    df['decision1'] = 0
    
    # I removed the first row because first row of vic is 00:00:00, 
    # which is the last period from the previous year
    df = df.drop([0], axis=0) 
    
    return df

In [5]:
def algorithm3(df):
    """ Finds optimal charge and discharge time across the dataset """
    
    for i in list(df.index):

        if ((i+LOOKAHEAD) < len(df)):

            thelist = list(df.iloc[i:i+LOOKAHEAD][PRICE])

            if (df.at[i,PRICE] <= min(thelist)):
                df.at[i,'decision1'] = CHARGE

            if (df.at[i,PRICE] >= max(thelist)):
                df.at[i,'decision1'] = DISCHARGE
        
        else:
            
            thelist = df.iloc[i:][PRICE]

            if (len(thelist) > 0):

                if (df.at[i,PRICE] <= min(thelist)):
                    df.at[i,'decision1'] = CHARGE

                if (df.at[i,PRICE] >= max(thelist)):
                    df.at[i,'decision1'] = DISCHARGE
            
    return df

In [6]:
def improve(df):
    """ This function tries to modify the decision by increasing the discharging frequency when it's idle """
    
    df['decision2'] = df['decision1']
    
    for i in range(1, len(df)-1):
        
        # if the current decision is idle
        if df.at[i, 'decision1'] == 0:
            prev = df.at[i-1, 'decision1']
            next_ = df.at[i+1, 'decision1']
            
            # find the next non-idle decision
            j = i+1
            while next_ == 0:
                j += 1
                next_ = df.at[j, 'decision1']
                
            # if the previous decision and the next non-idle decision is the same
            if next_ == prev:
                prices = list(df.loc[i:j-1, PRICE])
                mean_price = sum(prices)/len(prices)
                
                # discharge when the mean price of the idle decisions is greater
                # by THRESHOLD_PRICE than the next non-idle decision
                if mean_price - df.at[j, PRICE] > THRESHOLD_PRICE:
                    df.loc[i:j-1, 'decision2'] = DISCHARGE
                
    
    return df

In [7]:
def get_opencap(i, df):
    """Get the opening battery capacity for every 30-minutes interval 
    Input:
        i : Current row in df
        df : DataFrame with 'opening' and 'closing' column
    Return 
        opening_cap : Opening battery capacity. Integer dtype
    
    """

    if i != 1: #Not the first row
        df.at[i,"opening"] = df.at[i-1,"closing"]
    opening_cap = math.ceil(df.at[i, "opening"])
    
    return opening_cap
    
def get_dispatch(rawPower):
    """ Get the power dispatched for every 30-minutes interval 
    Input:
        rawPower : Current rawPower. Integer Datatype
    Return:
        dispatch : Raw_power dispatched from the market. Integer dtype
    """
    if rawPower < 0:
        eff = 1
    else:
        eff = DISCHARGE_EFF / 100
            
    dispatch = round(rawPower / 2 * eff, 0)
    return dispatch

def get_closecap(opening_cap, dispatch):
    """Get the battery closing capacity for every 30-minutes interval
    Input:
        opening_cap : Opening battery capacity. Integer dtype
        dispatch : Raw_power dispatched from the market. Integer dtype
    Return:
        closecap : Closing battery capacity. Integer dtype
    """
    if dispatch < 0:
        eff = CHARGE_EFF / 100
    else:
        eff = 100 / DISCHARGE_EFF

    closecap = math.ceil(max(0, min((opening_cap - (dispatch * eff)), CAPACITY)))
    return closecap
    
def get_revenue(price, dispatch):
    """ Get the Revenue for every 30-minutes interval
    Input:
        price : Market spot price for electricity. Float dtype
        dispatch : Raw_power dispatched from the market. Integer dtype
    Return:
        revenue : current revenue. Integer dtype
    """
    if dispatch < 0:
        factor = 1/MLF
    else:
        factor = MLF 
    revenue = math.ceil(price * dispatch * factor)
    return revenue

def run_algo3(i, df, opening_cap):
    """ Get the Raw Power for every 30-minutes interval
    Input:
        i : Current row in df
        df : DataFrame with 'charge_forecaset' and 'discharge_forecast' column
    Return:
        opening_cap : Opening battery capacity. Integer dtype
    """
    raw_power = 0
    if (df.at[i,'decision2'] == CHARGE):
        raw_power = -min(POWER, (CAPACITY-opening_cap)/EFFICIENCY*2)     # CHARGE
        
    elif (df.at[i,'decision2'] == DISCHARGE):
        raw_power = min(POWER, opening_cap*2)      # DISCHARGE
            
    return raw_power

def calculate(df):
    """ Calculate the Battery Opening, Closing Capacity, Raw Power, Market dispatch 
        and Revenue for the entire df.
    Input:
        df : DataFrame with 'price, opening', 'closing', 'raw_power', 'dispatch'
            and 'revenue' column
    Return :
        df : DataFrame with 'price, opening', 'closing', 'raw_power', 'dispatch'
            and 'revenue' column
    """
    # Go through each 30-minute interval of df
    for i in list(df.index):
        # get current Spot Price
        price = df.at[i, PRICE]
        
        # update opening capacity
        opening_cap = get_opencap(i, df)

        # find raw_power
        rawPower = run_algo3(i, df, opening_cap)
        df.at[i,'raw_power'] = rawPower

        # find market_dispatch 
        dispatch = get_dispatch(rawPower)
        df.at[i,"dispatch"] = dispatch

        # find closing_capacity   
        df.at[i,"closing"] = get_closecap(opening_cap, dispatch)

        #find revenue        
        df.at[i,"revenue"] = get_revenue(price, dispatch)
    
    return df

In [8]:
def show_result(df):
    """ Print the revenue related information computed from df.
    Input: 
        df : DataFrame with 'revenue' column.
    """
    print("Total revenue in the dataset:", df["revenue"].sum())
    print("Total days in the dataset:", len(df)/48)
    print("Revenue per day:", df["revenue"].sum() / (len(df)/48))
    
    return None

def run_all(ori_df):
    """ Run the entire data pipeline including initialisng (Data Processing), 
    finding the optimal charging and discarging period (Data Modelling), 
    calculating the revenue based on the optimal period mentioned above (Model Testing and Evaluation).
    
    Input:
        ori_df : DataFrame which contains spot_price for every 30-minute interval. 
    Return:
        df : DataFrame with 'price, opening', 'closing', 'raw_power', 'dispatch'
            and 'revenue' column.
    """
    # Start time
    start = time.time()
    # Initialise df
    df2 = create_df(ori_df)
    
    # Find Optimal Charging and Discharging period
    df3 = algorithm3(df2)
    
    df3 = improve(df3)
        
    # Calculate the revenue
    df = calculate(df3)
    # Show the revenue       
    show_result(df)
    # End Time      
    end = time.time()
    print("Time Complexity for running the entire Algorithm 3: {time_taken}s".format(time_taken = end-start))
            
    return df

In [9]:
# Highest Revenue when Lookahead == 5 without improve function
# 75829.73/day <- lookahead == 5

# with improve function -> revenue at max when lookahead == 4, revenue == 72452.81/day

for i in list(np.arange(1, 16, 1)):
    print("Lookahead is:", i)
    THRESHOLD_PRICE = 70
    LOOKAHEAD = i
    test_run = run_all(vic_data)
    print()

Lookahead is: 1
Total revenue in the dataset: 59578192
Total days in the dataset: 1322.0
Revenue per day: 45066.71104387292
Time Complexity for running the entire Algorithm 3: 10.187882900238037s

Lookahead is: 2
Total revenue in the dataset: 85492775
Total days in the dataset: 1322.0
Revenue per day: 64669.270045385776
Time Complexity for running the entire Algorithm 3: 10.024730920791626s

Lookahead is: 3
Total revenue in the dataset: 91610910
Total days in the dataset: 1322.0
Revenue per day: 69297.20877458397
Time Complexity for running the entire Algorithm 3: 10.627951860427856s

Lookahead is: 4
Total revenue in the dataset: 92397129
Total days in the dataset: 1322.0
Revenue per day: 69891.92813918306
Time Complexity for running the entire Algorithm 3: 12.252194166183472s

Lookahead is: 5
Total revenue in the dataset: 91759182
Total days in the dataset: 1322.0
Revenue per day: 69409.36611195159
Time Complexity for running the entire Algorithm 3: 11.487606048583984s

Lookahead is: 